colonne pas utilisé pour le moment : 'destination_IP_address', 'source_IP_address' start_time

In [1]:
import pandas as pd
import numpy as np

df_clean = pd.read_csv("data/clean_20150101.csv", sep = "\t")
np.shape(df_clean)

columns_name = ['duration_connection','service','source_bytes','destination_bytes','count','same_srv_rate','serror_rate','srv_serror_rate','dst_host_count','dst_host_srv_count','dst_host_same_src_port_rate','dst_host_serror_rate','dst_host_srv_serror_rate','flag','IDS_detection','malware_detection','ashula_detection','label','source_IP_address','source_port_number','destination_IP_address','destination_port_number','start_time','duration']
df_clean.columns = columns_name

df_clean = df_clean.drop(['destination_IP_address', 'source_IP_address', 'start_time'], axis = 1)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

list_encode_columns = ["service", "flag", "malware_detection", "ashula_detection", "duration", "IDS_detection"]

for col in list_encode_columns:
    df_clean[col] = le.fit_transform(df_clean[col])
    
print(df_clean.head())

   duration_connection  service  source_bytes  destination_bytes  count  \
0             0.000000        2             0                  0      0   
1             0.000000        2             0                  0      0   
2             0.000000        2             0                  0      0   
3             0.000000        2             0                  0      0   
4             0.000052        2             0                  0      0   

   same_srv_rate  serror_rate  srv_serror_rate  dst_host_count  \
0            0.0          0.0              0.0               0   
1            0.0          0.0              0.0               0   
2            0.0          0.0              0.0               0   
3            0.0          0.0              0.0               0   
4            0.0          0.0              0.0               0   

   dst_host_srv_count  ...  dst_host_serror_rate  dst_host_srv_serror_rate  \
0                   0  ...                   0.0                       0.0

In [2]:
df_clean.loc[df_clean['malware_detection'] != 0] = 1
df_clean

,duration_connection,service,source_bytes,destination_bytes,count,same_srv_rate,serror_rate,srv_serror_rate,dst_host_count,dst_host_srv_count,...,dst_host_serror_rate,dst_host_srv_serror_rate,flag,IDS_detection,malware_detection,ashula_detection,label,source_port_number,destination_port_number,duration
0,0.000000,2,0,0,0,0.0,0.0,0.00,0,0,...,0.0,0.0,6,0,0,0,-1,47904,23,1
1,0.000000,2,0,0,0,0.0,0.0,0.00,0,0,...,0.0,0.0,6,0,0,0,-1,58974,23,1
2,0.000000,2,0,0,0,0.0,0.0,0.00,0,0,...,0.0,0.0,6,0,0,0,-1,37174,23,1
3,0.000000,2,0,0,0,0.0,0.0,0.00,0,0,...,0.0,0.0,6,0,0,0,-1,40711,3389,1
4,0.000052,2,0,0,0,0.0,0.0,0.00,0,0,...,0.0,0.0,10,3,0,0,-1,8429,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381099,0.000585,0,44,104,10,1.0,0.0,0.00,84,99,...,0.0,0.0,9,0,0,0,-1,27124,53,2
381100,0.000784,0,44,104,11,1.0,0.0,0.00,84,99,...,0.0,0.0,9,0,0,0,-1,56495,53,2
381101,0.418711,2,20,905,1,0.0,0.0,1.00,24,0,...,0.0,0.0,7,0,0,0,-1,53952,22,1
381102,0.437801,2,20,724,1,0.0,0.0,0.80,26,0,...,0.0,0.0,7,0,0,0,1,54063,22,1


In [3]:
list_onehot_columns = ["service", "flag", "ashula_detection", "duration", "IDS_detection"]
for column_one_hot in list_onehot_columns:  
    one_hot = pd.get_dummies(df_clean[column_one_hot], prefix = column_one_hot)
    # Drop column B as it is now encoded
    df_clean = df_clean.drop(column_one_hot,axis = 1)
    # Join the encoded df
    df_clean = df_clean.join(one_hot)

In [4]:
df_clean = df_clean.astype(int)

# algo

In [10]:
X = df_clean.drop(['malware_detection'], axis = 1)
y = df_clean[['malware_detection']]

X=X.values.reshape((X.values.shape[0], X.values.shape[1], 1))
print("X size :", np.shape(X))
print("y size :", np.shape(y))

X size : (381104, 107, 1)
y size : (381104, 1)


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [16]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout
from keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

model=Sequential()

model.add(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.2), input_shape=(107,1), activation='relu'))
model.add(LSTM(32, activation='relu', kernel_regularizer=l2(0.2)))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])


# train the model 
h = model.fit(X_train, y_train,validation_data = (X_test, y_test), epochs=5)

Epoch 1/5
9528/9528 [==============================] - 578s 61ms/step - loss: 15.4736 - accuracy: 4.9199e-04 - val_loss: 15.2418 - val_accuracy: 5.1167e-04
Epoch 2/5
9528/9528 [==============================] - 513s 54ms/step - loss: 15.2424 - accuracy: 4.9199e-04 - val_loss: 15.2418 - val_accuracy: 5.1167e-04
Epoch 3/5
9528/9528 [==============================] - 464s 49ms/step - loss: 15.2424 - accuracy: 4.9199e-04 - val_loss: 15.2418 - val_accuracy: 5.1167e-04
Epoch 4/5
9528/9528 [==============================] - 466s 49ms/step - loss: 15.2424 - accuracy: 4.9199e-04 - val_loss: 15.2418 - val_accuracy: 5.1167e-04
Epoch 5/5
9528/9528 [==============================] - 481s 51ms/step - loss: 15.2424 - accuracy: 4.9199e-04 - val_loss: 15.2418 - val_accuracy: 5.1167e-04
